In [1]:
#The purpose of this notebook is to test some hypotheses the authors had regarding multi-issue distributions
# This notebook was used to create the tables for multi-issue distributions in the paper, but was used mainly for testing purposes.


#Imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import numpy as np
import pacal as pa
from mpl_toolkits.mplot3d import Axes3D  
from skimage import measure
%matplotlib inline

from astropy.table import QTable, Table, Column
from astropy import units as u
import astropy as astropy
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

mean = 5
 

Using compiled interpolation routine
Using compiled sparse grid routine


C:\Users\University Vivikth\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def sorted_k_partitions(seq, k):
    """Returns a list of all unique k-partitions of `seq`.

    Each partition is a list of parts, and each part is a tuple.

    The parts in each individual partition will be sorted in shortlex
    order (i.e., by length first, then lexicographically).

    The overall list of partitions will then be sorted by the length
    of their first part, the length of their second part, ...,
    the length of their last part, and then lexicographically.
    """
    n = len(seq)
    groups = []  # a list of lists, currently empty

    def generate_partitions(i):
        if i >= n:
            yield list(map(tuple, groups))
        else:
            if n - i > k - len(groups):
                for group in groups:
                    group.append(seq[i])
                    yield from generate_partitions(i + 1)
                    group.pop()

            if len(groups) < k:
                groups.append([seq[i]])
                yield from generate_partitions(i + 1)
                groups.pop()

    result = generate_partitions(0)

    # Sort the parts in each partition in shortlex order
    #result = [sorted(ps, key = lambda p: (len(p), p)) for ps in result]
    # Sort partitions by the length of each part, then lexicographically.
    #result = sorted(result, key = lambda ps: (*map(len, ps), ps))
    result = [ps for ps in result]
    return result

#Finds global optimum pooling arrangement for a given list of distributions
def global_optimum_finder(distributions):

    max_split_value = 0
    for indx, dist in enumerate(distributions):
        Combinations = sorted_k_partitions(distributions, indx + 1)
        max_combo_val = 0 #Make a best combination tuple.
        for combo in Combinations:
            #agree
            current_combo_val = 0
            for agreement in combo:
                    currrent_agreement_val = 0
                    for andx, issue in enumerate(agreement):
                        if andx == 0:
                            agr_pool = issue
                        else:
                            agr_pool += issue
                    agr_pool_pos = agr_pool | pa.Gt(0)
                    current_agreement_val = agr_pool.ccdf_value(0) * agr_pool_pos.mean() * agr_pool.ccdf_value(0)
                    current_combo_val += current_agreement_val
            if current_combo_val > max_combo_val:
                max_combo = combo
                max_combo_val = current_combo_val
                #print(current_combo_val, combo, '\n')
        if max_combo_val > max_split_value:
            max_split_value = max_combo_val
            best_split_combo = max_combo
            
    return best_split_combo, max_split_value
                #Can figure out expected value of each agreement.
                #Sum to get value of combo.
                #Pick best combo.
                #Want to store Combo.

#Determins the support of a distribution
def Support_Getter(dist):
    if type(dist) ==  pa.standard_distr.UniformDistr:
        return np.sqrt(3 * dist.var())
    elif type(dist) == pa.standard_distr.NormalDistr:
        return dist.std()

def pretty_table(optimal_agreement_list):
    t = Table()

    for indx, agreement in enumerate(optimal_agreement_list):
        supports = [Support_Getter(issue) for issue in optimal_agreement_list[indx]]
        string = ' '
        for s in supports:
            string += (str(s) + ' , ')
        t['Agreement ' + str(indx) ] = [string]

    return t

In [3]:

obj = pa.NormalDistr(5, 0.15)
obj2 = pa.NormalDistr(5, 0.15)
obj3 = pa.NormalDistr(5, 8)
obj4 = pa.NormalDistr(5, 8)


disti = [obj, obj2, obj3, obj4]

optimal_agreement_list = global_optimum_finder(disti)[0]
pretty_table(optimal_agreement_list)

Agreement 0
str85
"0.14999999999999997 , 0.14999999999999997 , 7.999999999999999 , 7.999999999999999 ,"


In [4]:
def col(s):
    obj = pa.UniformDistr(5 - s, 5 + s)
    obj2 = pa.UniformDistr(5 - s, 5 + s)
    obj3 = pa.UniformDistr(5 - s, 5 + s)
    obj4 = pa.UniformDistr(5 - s, 5 + s)


    disti = [obj, obj2, obj3, obj4]

    optimal_agreement_list = global_optimum_finder(disti)[0]
    t = pretty_table(optimal_agreement_list)
    return len(t.colnames)

In [5]:
obj3 = pa.NormalDistr(5, 0.15)
obj4 = pa.NormalDistr(5, 0.15)
global_optimum_finder([obj3, obj4])[0]

[(Normal(5,0.15)#1831440091592, Normal(5,0.15)#1831440092872)]

In [6]:
obj = pa.UniformDistr(5 - 15.2, 5 + 15.2)
obj2 = pa.UniformDistr(5 - 5.1, 5 + 5.1)
obj3 = pa.UniformDistr(5 - 5.2, 5 + 5.2)
obj4 = pa.UniformDistr(5 - 15.3, 5 + 15.3)
obj5 = pa.UniformDistr(5 - 30, 5 + 30)


disti = [obj, obj2, obj3, obj4, obj5]

optimal_agreement_list = global_optimum_finder(disti)[0]
pretty_table(optimal_agreement_list)

Agreement 0,Agreement 1,Agreement 2
str29,str27,str8
"15.199999999999996 , 15.3 ,","5.1 , 5.199999999999999 ,","30.0 ,"
